In [1]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import sqlite3 as sl
import os

In [2]:
%load_ext autoreload
%autoreload 2

## Inspect AST Manually

In [3]:
import ast
import astpretty

In [8]:
def show_tree(code: str):
    tree = ast.parse(code)
    astpretty.pprint(tree)

In [10]:
show_tree("""
from qiskit import transpile
qc = transpile(qc, basis_gates=None, optimization_level=0)
""")

Module(
    body=[
        ImportFrom(
            lineno=2,
            col_offset=0,
            end_lineno=2,
            end_col_offset=28,
            module='qiskit',
            names=[alias(name='transpile', asname=None)],
            level=0,
        ),
        Assign(
            lineno=3,
            col_offset=0,
            end_lineno=3,
            end_col_offset=58,
            targets=[Name(lineno=3, col_offset=0, end_lineno=3, end_col_offset=2, id='qc', ctx=Store())],
            value=Call(
                lineno=3,
                col_offset=5,
                end_lineno=3,
                end_col_offset=58,
                func=Name(lineno=3, col_offset=5, end_lineno=3, end_col_offset=14, id='transpile', ctx=Load()),
                args=[Name(lineno=3, col_offset=15, end_lineno=3, end_col_offset=17, id='qc', ctx=Load())],
                keywords=[
                    keyword(
                        arg='basis_gates',
                        value=Constant(lineno=

In [12]:
show_tree("""
from qiskit import transpile
qc = transpile(qc, basis_gates=['rx', 'ry', 'rz', 'h', 'cx'], optimization_level=0)
""")

Module(
    body=[
        ImportFrom(
            lineno=2,
            col_offset=0,
            end_lineno=2,
            end_col_offset=28,
            module='qiskit',
            names=[alias(name='transpile', asname=None)],
            level=0,
        ),
        Assign(
            lineno=3,
            col_offset=0,
            end_lineno=3,
            end_col_offset=83,
            targets=[Name(lineno=3, col_offset=0, end_lineno=3, end_col_offset=2, id='qc', ctx=Store())],
            value=Call(
                lineno=3,
                col_offset=5,
                end_lineno=3,
                end_col_offset=83,
                func=Name(lineno=3, col_offset=5, end_lineno=3, end_col_offset=14, id='transpile', ctx=Load()),
                args=[Name(lineno=3, col_offset=15, end_lineno=3, end_col_offset=17, id='qc', ctx=Load())],
                keywords=[
                    keyword(
                        arg='basis_gates',
                        value=List(
          

## Change Target Basis

In [35]:
from qmt import get_mr_function_and_kwargs

In [32]:
from metamorph import *

In [33]:
config = load_config_and_check("../config/qmt_v01.yaml")

In [50]:
morph, kwargs = get_mr_function_and_kwargs(config, metamorphic_strategy="change_basis")
print(morph("""
# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)

# SECTION
# NAME: OPTIMIZATION_LEVEL

from qiskit import transpile
qc = transpile(qc, basis_gates=None, optimization_level=0)
""", **kwargs))

Follow: gateset replaced with:  ['rx', 'ry', 'rz', 'p', 'cx']
# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)
# SECTION
# NAME: OPTIMIZATION_LEVEL
from qiskit import transpile
qc = transpile(qc, basis_gates=['rx', 'ry', 'rz', 'p', 'cx'],
    optimization_level=0)



## Change Optimization Level

In [48]:
morph, kwargs = get_mr_function_and_kwargs(config, metamorphic_strategy="change_opt_level")
print(morph("""
# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)

# SECTION
# NAME: OPTIMIZATION_LEVEL

from qiskit import transpile
qc = transpile(qc, basis_gates=None, optimization_level=0)
""", **kwargs))

Follow: optimization level changed: 0 -> 1
# SECTION
# NAME: OPTIMIZATION_PASSES

from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import *
passmanager = PassManager()
qc = passmanager.run(qc)
# SECTION
# NAME: OPTIMIZATION_LEVEL
from qiskit import transpile
qc = transpile(qc, basis_gates=1, optimization_level=0)

